In [1]:
import pandas as pd 
import numpy as np 

In [3]:
#Load dataset
filename1 = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\tcga_cli.txt"
filename2 = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\tcga_rna.txt"
datac = pd.read_csv(filename1, sep=';')
datar = pd.read_csv(filename2, sep=';')

In [7]:
datar[1:10].head()

,patients,g100130426,g100133144,g100134869,g10357,g10431,g136542,g155060,g26823,g280660,...,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR,patients2
73,TCGA-22-A5C4,0.0,101.50,43.50,269.56,2246,0,667,25,0,...,1082,3180,712,2658.0,9258.0,2631,2106,48,0,TCGA-22-A5C4-01A-12R-A27Q-07
74,TCGA-2A-A8VL,0.0,40.30,33.70,157.13,2546,0,612,0,0,...,552,2139,74,1344.0,4474.0,2891,981,387,0,TCGA-2A-A8VL-01A-21R-A37L-07
75,TCGA-2A-A8VO,0.0,29.07,20.93,93.40,1300,0,569,0,0,...,577,1366,252,1138.0,4081.0,1285,964,439,0,TCGA-2A-A8VO-01A-11R-A37L-07
76,TCGA-2A-A8VT,0.0,132.97,161.03,256.83,1585,0,1985,3,0,...,1629,4699,121,3239.0,4174.0,5661,3736,98,1,TCGA-2A-A8VT-01A-11R-A37L-07
77,TCGA-2A-A8VV,0.0,46.18,34.82,229.79,1952,0,504,0,0,...,895,2159,137,1302.0,6500.0,2188,2229,45,0,TCGA-2A-A8VV-01A-11R-A37L-07


In [6]:
datac.head()

,patients,gender,race,ethnicity,vital_status,tumor_status,new_tumor_event_dx_indicator,abr,aux
36,TCGA-21-A5DI,MALE,WHITE,NOT HISPANIC OR LATINO,Alive,[Unknown],NO,LUSC,1
73,TCGA-22-A5C4,MALE,WHITE,NOT HISPANIC OR LATINO,Alive,TUMOR FREE,NO,LUSC,1
74,TCGA-2A-A8VL,MALE,[Not Available],[Not Available],Alive,TUMOR FREE,NO,PRAD,1
75,TCGA-2A-A8VO,MALE,[Not Available],[Not Available],Alive,TUMOR FREE,NO,PRAD,1
76,TCGA-2A-A8VT,MALE,[Not Available],[Not Available],Alive,TUMOR FREE,NO,PRAD,1
